## Задание
### Создание чат-бота для определения текущего времени и погоды в определённый день

## Введение
### Необходимо написать чат-бота на основе LLM, который:
* Отвечает на запросы пользователя о текущем времени
* Определяет погоду по запросу за указанную дату
* Использует сторонние API для получения информации (например, `Open Meteo`)


## Требования к реализации:
* Чат-бот должен корректно определять дату из сообщений Пользователя
* Результаты выполнения кода и пример диалога должны быть отображены в ячейках
* Можно использовать консольный интерфейс через `input` и `print`

In [ ]:
%%capture
!pip install langchain langchain-community

In [ ]:
# Импорты и установка окружения
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.llms import HuggingFaceEndpoint
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

# Подавление предупреждений
import warnings
warnings.simplefilter('ignore', FutureWarning)

import os
import requests
from datetime import datetime

os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'token'

llm = HuggingFaceEndpoint(repo_id='mistralai/Mistral-Nemo-Instruct-2407', temperature=0.1)

# API для определения температуры
url = 'https://archive-api.open-meteo.com/v1/archive'

# Координаты Москвы
params = {
    'latitude': 55.7558,
    'longitude': 37.6173,
    'hourly': 'temperature_2m',
    'timezone': 'Europe/Moscow'
    }

# Функции бота
def get_current_time():
    """
    Функция для получения текущего времени
    """
    return str(datetime.now().strftime("%H:%M"))

def get_weather_for_date(date):
    """
    Функция для получения средней температуры в указанный день
    """
    date = date.replace('</s>', '')
    params['start_date'] = str(date.split('\n')[0])
    params['end_date'] = str(date.split('\n')[0])

    response = requests.get(url, params=params)
    data = response.json()

    # Расчет средней температуры
    temps = data['hourly']['temperature_2m']
    avg_temp = round(sum(temps) / len(temps), 1)

    return avg_temp

def get_date_of_interest():
    """
    Функция анализа истории чата и определения даты из последнего сообщения
    """
    user_history = '\n'.join([msg.content for msg in history.messages])

    prompt = f"""
    Внимательно прочитай чат пользователя с ботом и определи, какая дата интересует его в последнем предложении.
    В ответ укажи только последнюю дату в формате YYYY-mm-dd.

    Чат с пользователем:
    {user_history}
    """

    answer = llm.invoke(prompt)
    return answer

def introduce_yourself():
    """
    Функция приветствия
    """
    system_prompt = """
    Ты - погодный чат-бот.
    Твоя задача кратко отвечать на вопросы пользователя.
    Вежливо поздоровайся с пользователем и предложи свою помощь в определении температуры.
    """
    return system_prompt

# Описание тулов для агента
tools = [
    StructuredTool.from_function(
        func=get_date_of_interest,  # The function to be wrapped as a tool
        name='Get More Information about the unknown Date',  # Name of the tool
        description='May be useful to get more information from the chat',  # Description of what the tool does
    ),
    StructuredTool.from_function(
        func=get_weather_for_date,  # The function to be wrapped as a tool
        name='Get Weather For Date',  # Name of the tool
        description='Полезно при определении температуры, когда уже определился с датой: Дата - строго в формате YYYY-mm-dd',  # Description of what the tool does
    ),
    StructuredTool.from_function(
        func=get_current_time,  # The function to be wrapped as a tool
        name='What Time is It',  # Name of the tool
        description='Полезно, чтобы узнать, сколько время',  # Description of what the tool does
    ),
    StructuredTool.from_function(
        func=introduce_yourself,  # The function to be wrapped as a tool
        name='Introduce Yourself',  # Name of the tool
        description='Используй, чтобы узнать, что ты умеешь и чем можешь помочь',  # Description of what the tool does
    )
]

# Создание агента
prompt = hub.pull('hwchase17/react')
agent = create_react_agent(llm, tools, prompt)

# Инициализация исполнителя агента
agent_executor = AgentExecutor(
    agent = agent,  # The agent to execute
    tools = tools,  # The tools available to the agent
    # verbose = True,  # Enable verbose output
    max_iterations = 4,  # Set maximum number of iterations
    early_stopping_method = 'force',  # Force stop after max_iterations
    handle_parsing_errors = True,
)

# Инициализация истории чата
history = ChatMessageHistory()

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
# Основной цикл работы чат-бота
while True:
    user_input = input('Вы: ')

    # Сценарий выхода из чата
    if 'пока' in user_input.lower():
        print('Погодный чат-бот: Пока!')
        break

    # Добавление сообщений пользователя в историю
    history.add_user_message(f'Вопрос Пользователя: {user_input}')

    # Получение ответа от агента
    result = agent_executor.invoke({'input': user_input})
    answer = result['output']

    # Сохранение ответа бота в историю
    history.add_ai_message(f'Твой предыдущий ответ: {answer}')

    # Вывод ответа бота в консоль
    print("Погодный чат-бот:", answer.replace('</s>', ''))

Вы: Привет! Что умеешь?
Погодный чат-бот: Привет! Я погодный чат-бот. Моя задача кратко отвечать на вопросы пользователя. Я могу помочь вам определить температуру. Как я могу вам помочь?
Вы: Какая погода была 12 мая 1991 года?
Погодный чат-бот: 12 мая 1991 года температура была 10.4°C.
Вы: А какой погода стала через месяц?
Погодный чат-бот: Через месяц, 12 июня 1991 года, температура была 18.2°C.
Вы: Сколько время?
Погодный чат-бот: 18:15
Вы: Спасибо, пока!
Погодный чат-бот: Пока!
